In [ ]:
import swat 

In [ ]:
#connection to viya

viya_conn=swat.CAS('your viya host.com',5570, 'username', 'password')

In [ ]:
from dlpy.transformers import bert_model

### STEP 1: CREATE BERT CLASSIFICATION MODEL
Note the viya_conn variable. This is the SWAT connection to the active Viya session referred
to earlier.

In [ ]:
cache_dir = 'cache_dir' #can be relative or absolute path. here the relative path is used. 
bert = bert_model.BERT_Model(viya_conn,
cache_dir,
'bert-base-uncased',
2,
num_hidden_layers=12,
max_seq_len=256,
verbose=True )

### STEP 2: PREPARE DATA

In [ ]:
import pandas as pd 
reviews = pd.read_csv('data/Reviews.csv', header=0, encoding='utf-8')

Assign a numeric value to indicate positive or negative sentiment for each review.
Filter out the neutral reviews (3 stars), and then assign a negative label to 1- and 2-star reviews and a
positive label to 4- and 5-star reviews:

In [ ]:
t_idx = reviews["Score"] != 3
inputs = reviews[t_idx]["Text"].to_list()
targets = reviews[t_idx]["Score"].to_list()
for ii,val in enumerate(targets):
 inputs[ii] = inputs[ii].replace("<br />","")
 if (val == 1) or (val == 2): # negative reviews
     targets[ii] = 1
 elif (val == 4) or (val == 5): # positive reviews
     targets[ii] = 2

Finally, import the data preparation helper function. Invoking the helper function tokenizes
the review data and creates the three input values (token, position, segment) associated
with each token as well as the sentiment target for each review.

In [ ]:
from dlpy.transformers.bert_utils import bert_prepare_data

In [ ]:
num_tgt_var, train, test = bert_prepare_data(viya_conn,
 bert.get_tokenizer(),
 input_a=inputs,
 target=targets,
 train_fraction=0.8,
 segment_vocab_size=bert.get_segment_size(),
 classification_problem=bert.get_problem_type(),
#  truncation=True,
 max_seq_len= 512,
 verbose=True)

### STEP 3: CREATE SAS VIYA BERT MODEL

The third step is to create a SAS Deep Learning model that is the equivalent of the base
BERT model plus a classification (fully connected) layer;

In [ ]:
bert.compile(num_target_var=num_tgt_var)

### STEP 4: ATTACH MODEL PARAMETERS

The fourth step is to attach the trained model parameters stored in the HDF5 file to the
BERT model.

In [ ]:
bert.load_weights('cache_dir/bert-base-uncased.kerasmodel.h5', num_target_var=num_tgt_var, freeze_base_model=False)

### STEP 5: FINE-TUNE BERT CLASSIFICATION MODEL

Perform fine-tuning training. 
When the fit() function finishes executing, your BERT model is fine-tuned for sentiment
analysis.

In [ ]:
bert.set_optimizer_parameters(learning_rate=2e-5)
bert.fit(train,
 data_specs= bert.get_data_spec(num_tgt_var),
 optimizer=bert.get_optimizer(),
 text_parms=bert.get_text_parameters(),
 seed=12345,
 n_threads=32)

### MODEL EVALUATION

You can now evaluate your fine-tuned sentiment analysis model by using the test data set.
Use the predict() function exposed by the DLPy BERT model object. 

In [ ]:
res = bert.predict(test,
 text_parms=bert.get_text_parameters())
print(res['ScoreInfo'])